# 7. Planner Pattern with Semantic Kernel

The planner pattern involves breaking down complex tasks into smaller, manageable steps. The planner creates a step-by-step plan, and then executes each step using available tools and agents, maintaining context throughout the process.

## Define Tools

We will first define the tools we want to use. For this example, we will use a web search tool to demonstrate the planner's ability to use external resources.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from typing import Annotated, List, Dict, Any
import json

Set up the kernel and service

In [ ]:
# Create a kernel instance
kernel = Kernel()

# Add Azure OpenAI chat completion service
service_id = "azure_openai"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
        deployment_name="gpt-4.1-mini",
    )
)

Define search and research tools

In [ ]:
class ResearchPlugin:
    """
    A plugin for research and information gathering
    """
    
    @kernel_function(
        description="Search for information on the web",
        name="web_search"
    )
    def web_search(
        self,
        query: Annotated[str, "The search query"]
    ) -> Annotated[str, "Search results"]:
        # Mock search results - in real implementation, you'd use a real search API
        mock_results = {
            "paris museums": "Top Paris Museums: Louvre Museum (€17), Musée d'Orsay (€16), Centre Pompidou (€15), Musée Rodin (€13), Musée de l'Orangerie (€12.50)",
            "euro to rupee": "Current exchange rate: 1 EUR = 92.50 INR (approximate)",
            "louvre museum": "The Louvre Museum is the world's largest art museum, home to the Mona Lisa. Entrance fee: €17",
            "musee dorsay": "Musée d'Orsay houses the world's finest collection of Impressionist art. Entrance fee: €16",
            "centre pompidou": "Centre Pompidou is known for modern art and unique architecture. Entrance fee: €15"
        }
        
        # Simple keyword matching for demo
        for key, result in mock_results.items():
            if key.lower() in query.lower():
                return f"Search results for '{query}': {result}"
        
        return f"Search results for '{query}': General information found."
    
    @kernel_function(
        description="Calculate currency conversion",
        name="currency_convert"
    )
    def currency_convert(
        self,
        amount: Annotated[float, "Amount to convert"],
        from_currency: Annotated[str, "Source currency (e.g., EUR)"],
        to_currency: Annotated[str, "Target currency (e.g., INR)"]
    ) -> Annotated[str, "Conversion result"]:
        # Mock exchange rates
        rates = {
            "EUR_INR": 92.50,
            "USD_INR": 83.12,
            "GBP_INR": 105.23
        }
        
        rate_key = f"{from_currency}_{to_currency}"
        if rate_key in rates:
            converted = amount * rates[rate_key]
            return f"{amount} {from_currency} = {converted:.2f} {to_currency}"
        
        return f"Exchange rate not available for {from_currency} to {to_currency}"

Create execution agent

In [ ]:
# Create execution kernel with research tools
execution_kernel = Kernel()
execution_kernel.add_service(AzureChatCompletion(service_id=service_id, deployment_name="gpt-4.1-mini"))
execution_kernel.add_plugin(ResearchPlugin(), plugin_name="research")

# Create execution agent
execution_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=execution_kernel,
    name="ExecutionAgent",
    instructions="""You are a helpful assistant that can execute tasks step by step.
    
Use the available tools to gather information and perform calculations as needed.
Be thorough and precise in your research and provide detailed results.""",
    execution_settings={
        service_id: execution_kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)
execution_agent.execution_settings[service_id].function_choice_behavior = FunctionChoiceBehavior.Auto()

Create planner agent

In [ ]:
# Create planner agent
planner_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=kernel,
    name="PlannerAgent",
    instructions="""You are a strategic planner that breaks down complex tasks into simple, actionable steps.
    
For the given objective, create a step-by-step plan. Each step should:
1. Be specific and actionable
2. Build upon previous steps
3. Include all necessary information
4. Lead towards the final answer

Respond with a JSON object in this format:
{
  "steps": [
    "Step 1 description",
    "Step 2 description",
    "Step 3 description"
  ]
}

Do not add superfluous steps. Make sure each step has all needed information.""",
    execution_settings={
        service_id: kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)

Create the main planner coordinator

In [ ]:
class PlanExecuteCoordinator:
    """
    Coordinates planning and execution of complex tasks
    """
    
    def __init__(self):
        self.planner = planner_agent
        self.executor = execution_agent
        self.reset_state()
    
    def reset_state(self):
        """Reset the planning state"""
        self.state = {
            "input": "",
            "plan": [],
            "past_steps": [],
            "response": ""
        }
    
    async def create_plan(self, objective: str) -> List[str]:
        """
        Create a step-by-step plan for the given objective
        """
        print(f"→ Creating plan for: {objective}")
        
        chat_history = ChatHistory()
        chat_history.add_user_message(objective)
        
        async for response in self.planner.invoke(chat_history):
            try:
                plan_data = json.loads(str(response.content))
                steps = plan_data.get("steps", [])
                
                print(f"→ Plan created with {len(steps)} steps:")
                for i, step in enumerate(steps, 1):
                    print(f"  {i}. {step}")
                
                return steps
            except json.JSONDecodeError:
                print(f"Failed to parse plan: {response.content}")
                return []
        
        return []
    
    async def execute_step(self, step: str, context: str = "") -> str:
        """
        Execute a single step of the plan
        """
        print(f"→ Executing step: {step}")
        
        chat_history = ChatHistory()
        
        # Add context from previous steps if available
        if context:
            chat_history.add_system_message(f"Previous context: {context}")
        
        chat_history.add_user_message(f"Execute this step: {step}")
        
        async for response in self.executor.invoke(chat_history):
            result = str(response.content)
            print(f"→ Step result: {result}")
            return result
        
        return "Step execution failed"
    
    async def execute_plan(self, objective: str) -> str:
        """
        Execute the complete plan for the given objective
        """
        self.reset_state()
        self.state["input"] = objective
        
        # Create plan
        plan = await self.create_plan(objective)
        if not plan:
            return "Failed to create a plan for the given objective."
        
        self.state["plan"] = plan
        
        # Execute each step
        context = ""
        for i, step in enumerate(plan, 1):
            print(f"\n=== Executing Step {i}/{len(plan)} ===")
            
            result = await self.execute_step(step, context)
            
            # Store the step execution
            step_info = (step, result)
            self.state["past_steps"].append(step_info)
            
            # Update context for next step
            context += f"Step {i}: {step} -> Result: {result}\n"
        
        # Generate final response
        final_response = await self.generate_final_response()
        self.state["response"] = final_response
        
        return final_response
    
    async def generate_final_response(self) -> str:
        """
        Generate a final response based on all executed steps
        """
        print("\n→ Generating final response...")
        
        chat_history = ChatHistory()
        
        # Summarize all steps and results
        summary = f"Original objective: {self.state['input']}\n\nSteps executed:\n"
        for i, (step, result) in enumerate(self.state["past_steps"], 1):
            summary += f"{i}. {step}\n   Result: {result}\n\n"
        
        chat_history.add_user_message(
            f"{summary}\nBased on all the steps executed above, provide a comprehensive final answer to the original objective."
        )
        
        async for response in self.executor.invoke(chat_history):
            return str(response.content)
        
        return "Failed to generate final response"
    
    def get_state_summary(self) -> Dict[str, Any]:
        """
        Get a summary of the current planning state
        """
        return {
            "objective": self.state["input"],
            "total_steps": len(self.state["plan"]),
            "completed_steps": len(self.state["past_steps"]),
            "remaining_steps": len(self.state["plan"]) - len(self.state["past_steps"]),
            "has_response": bool(self.state["response"])
        }

Test the planner with a simple task

In [ ]:
# Initialize the planner
planner = PlanExecuteCoordinator()

# Test with a simple objective
simple_objective = "Give me 5 must-see museums in Paris"
print(f"Testing with objective: {simple_objective}\n")

result = await planner.execute_plan(simple_objective)

print("\n" + "="*50)
print("FINAL RESULT:")
print("="*50)
print(result)

Test the planner with a complex multi-step task

In [ ]:
# Test with a more complex objective
complex_objective = "Find the 5 must-see museums in Paris and tell me their entrance fees in Indian Rupees"
print(f"\nTesting with complex objective: {complex_objective}\n")

result = await planner.execute_plan(complex_objective)

print("\n" + "="*50)
print("FINAL RESULT:")
print("="*50)
print(result)

Inspect the planner state

In [ ]:
# Show the planner state
print("\n=== Planner State Summary ===")
state_summary = planner.get_state_summary()
for key, value in state_summary.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

print("\n=== Detailed Step Execution ===")
for i, (step, result) in enumerate(planner.state["past_steps"], 1):
    print(f"\nStep {i}:")
    print(f"  Task: {step}")
    print(f"  Result: {result[:100]}{'...' if len(result) > 100 else ''}")

## Custom Planning Example

Test the planner's ability to handle different types of tasks

In [ ]:
# Test various objectives
test_objectives = [
    "Plan a 3-day itinerary for Paris focusing on art and culture",
    "Compare the costs of visiting 3 major museums in Paris vs London",
    "Research the best time to visit Paris museums to avoid crowds"
]

for i, objective in enumerate(test_objectives, 1):
    print(f"\n{'='*60}")
    print(f"TEST {i}: {objective}")
    print(f"{'='*60}")
    
    # Just show the planning step for these examples
    plan = await planner.create_plan(objective)
    
    print(f"\n→ Generated plan:")
    for j, step in enumerate(plan, 1):
        print(f"  {j}. {step}")

## Key Differences from LangChain Planner

1. **Agent-Based Architecture**: Uses dedicated `ChatCompletionAgent` instances for planning and execution rather than function-based approaches.

2. **Simplified State Management**: Uses a simple dictionary-based state instead of complex TypedDict structures with operators.

3. **Plugin Integration**: Execution agent has access to specialized plugins for different types of tasks.

4. **Flexible Tool Usage**: Tools are automatically discovered and used through Semantic Kernel's function choice behavior.

5. **Context Preservation**: Maintains context across step executions without complex graph state management.

6. **Clear Separation**: Planning and execution are clearly separated with distinct agents having specific roles.

7. **JSON-Based Communication**: Uses structured JSON for plan representation and parsing.

### Benefits:

- **Modularity**: Planner and executor are independent and can be modified separately
- **Extensibility**: Easy to add new tools and capabilities to the execution agent
- **Transparency**: Clear visibility into planning steps and execution results
- **Flexibility**: Can handle various types of objectives and planning scenarios
- **Maintainability**: Simple state management and clear control flow

### Use Cases:

- **Research Tasks**: Breaking down complex research into manageable steps
- **Data Analysis**: Planning and executing multi-step data analysis workflows
- **Trip Planning**: Creating detailed itineraries with multiple components
- **Project Management**: Breaking down projects into actionable tasks
- **Problem Solving**: Systematically approaching complex problems

This approach provides a clean, scalable way to implement planning and execution patterns that can handle complex, multi-step tasks while maintaining clarity and control throughout the process."